In [33]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from datetime import datetime
import os
import xlrd
import pandas as pd
import re

In [84]:
#import sys
#!{sys.executable} -m pip install selenium

# Code to generate State level data...
1. Column: "State","Confirmed","Active","Recovery","Deceased"
2. Source: https://www.covid19india.org/

In [85]:
driver = webdriver.Chrome("C:/Users/mangl/Desktop/python_projects/project_2/chromedriver.exe")
driver.get('https://www.covid19india.org/')
time.sleep(3)

states = driver.find_elements_by_xpath('//tbody[1]//span[@class="actual__title-wrapper"]')
##confirmed = driver.find_elements_by_xpath('//tr[@class="state"]/td[2]')
confirmed = driver.find_elements_by_xpath('//tr[@class ="state  " or @class ="state  is-odd" ]/td[2]/span[@class = "table__count-text"]')
span = driver.find_elements_by_xpath('//tr[@class ="state  " or @class ="state  is-odd" ]/td[2]/span')
actives = driver.find_elements_by_xpath('//tr[@class ="state  " or @class ="state  is-odd" ]/td[3]')
##recoveries = driver.find_elements_by_xpath('//tr[@class="state"]/td[4]')
recoveries = driver.find_elements_by_xpath('//tr[@class ="state  " or @class ="state  is-odd" ]/td[4]/span[@class = "table__count-text"]')
##deaths = driver.find_elements_by_xpath('//tr[@class="state"]/td[5]')
deaths = driver.find_elements_by_xpath('//tr[@class ="state  " or @class ="state  is-odd" ]/td[5]/span[@class = "table__count-text"]')
column = ["State","Confirmed","Active","Recovery","Deceased"]

res = pd.DataFrame(columns=column)

for state,confirm,span,active,recovered,death in zip(states,confirmed,span,actives,recoveries,deaths):
    #print(state.text, confirm.text.replace(span.text,''),active.text,recovered.text,death.text)
    State,Confirmed,Active,Recovery,Deceased = state.text, confirm.text.replace(span.text,''),active.text,recovered.text,death.text
    data = {"State":[State],"Confirmed":[Confirmed],"Active":[Active],"Recovery":[Recovery],"Deceased":[Deceased]}
    temp = pd.DataFrame(data=data)
    #print(temp)
    res = res.append(temp, ignore_index=True)
    
print("Success")
driver.close()
time_str = datetime.now().strftime('%Y%m%d%H%M%S')
name = "covid19_"+time_str+'.csv'
res.to_csv(name,index=False)
print("{} is generated in path {}".format(name,os.getcwd()))

Success
covid19_20200422174042.csv is generated in path C:\Users\mangl\Desktop\python_projects\project_2


# Code to generate District level data...
1. Columns: "STATE","DISTRICT","CONFIRMED"
2. Source: https://www.covid19india.org/

In [86]:
driver = webdriver.Chrome("C:/Users/mangl/Desktop/python_projects/project_2/chromedriver.exe")
driver.get('https://www.covid19india.org/')
time.sleep(3)
xpath = '//tbody[1]/tr'
loc = driver.find_elements_by_xpath(xpath)
state_regex = r">{1}([A-Za-z\s\*\.\d]+)<"
count_regex = r"\"table__count-text\">([\d\-\,]+)</span>"
dataset_dist = pd.DataFrame(columns=["STATE","DISTRICT","CONFIRMED"])

for ele in loc: 
    cls = (ele.get_attribute("class"))
    if cls.strip() in ("state","state  is-odd"):
        state = ele.text.split('\n')[0]
    elif cls.strip() in ("district","district is-odd") :
        test_str = ele.get_attribute("innerHTML")
        district = re.findall(state_regex, test_str, re.MULTILINE)[0]
        count = re.findall(count_regex, test_str, re.MULTILINE)[0]
        #print(state,district,count,sep=" : ")
        record = {"STATE":state, "DISTRICT":district, "CONFIRMED":count}
        dataset_dist = dataset_dist.append(record, ignore_index=True)
print("Success")
driver.close()

time_str = datetime.now().strftime('%Y%m%d%H%M%S')
name = "covid19_districtwise"+time_str+'.csv'
dataset_dist.to_csv(name,index=False)
print("{} is generated in path {}".format(name,os.getcwd()))

Success
covid19_districtwise20200422174142.csv is generated in path C:\Users\mangl\Desktop\python_projects\project_2


# Get the coordinates of districts from Google:
1. Columns: "STATE","DISTRICT","COORDINATES"
2. Source:https://www.google.com/

In [55]:
driver = webdriver.Chrome("C:/Users/mangl/Desktop/python_projects/project_2/chromedriver.exe")
driver.implicitly_wait(10) 
driver.get('https://www.google.com/')
xpath_search = '//input[@class = "gLFyf gsfi"]'
#search = driver.find_element_by_xpath(xpath_search)
xpath_click = '//div[@class="FPdoLc tfB0Bf"]//input[@class="gNO89b"]'
#click = driver.find_element_by_xpath(xpath_click)
xpath_coordinates = '//div[@class = "Z0LcW"]'
dataset_coordinates = pd.DataFrame(columns=["STATE","DISTRICT","COORDINATES"])

for place in dataset_dist.itertuples():
    #print(place.STATE,',', place.DISTRICT, 'Coordinates')
    try : 
    #print(place.STATE,',', place.DISTRICT, 'Coordinates')
        search = driver.find_elements_by_xpath(xpath_search)
        search = search[0]
        search.clear()
        search.send_keys(place.STATE,',', place.DISTRICT, ' Coordinates')
        click = driver.find_element_by_xpath(xpath_click)
        click.click()
        coordinates = driver.find_element_by_xpath(xpath_coordinates).text
        #print(place.STATE,',', place.DISTRICT, 'Coordinates: ',coordinates)
        record = {"STATE":place.STATE, "DISTRICT":place.DISTRICT, "COORDINATES":coordinates}
        dataset_coordinates = dataset_coordinates.append(record, ignore_index=True)
        driver.back()
    except NoSuchElementException:
        record = {"STATE":place.STATE, "DISTRICT":place.DISTRICT, "COORDINATES":''}
        dataset_coordinates = dataset_coordinates.append(record, ignore_index=True)
        driver.back()
        #print("No element found")

driver.close()
time_str = datetime.now().strftime('%Y%m%d%H%M%S')
name = "district_coordinates"+time_str+'.csv'
dataset_coordinates.to_csv(name,index=False)
print("{} is generated in path {}".format(name,os.getcwd()))

No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
No element found
district_coordinates20200422170946.csv is generated in path C:\Users\mangl\Desktop\python_projects\project_2


# Generates incremental records for District coordinates:
1. Columns: "STATE","DISTRICT","COORDINATES"
2. Source: https://www.google.com/

In [89]:
driver = webdriver.Chrome("C:/Users/mangl/Desktop/python_projects/project_2/chromedriver.exe")
driver.implicitly_wait(10) 
driver.get('https://www.google.com/')
xpath_search = '//input[@class = "gLFyf gsfi"]'
#search = driver.find_element_by_xpath(xpath_search)
xpath_click = '//div[@class="FPdoLc tfB0Bf"]//input[@class="gNO89b"]'
#click = driver.find_element_by_xpath(xpath_click)
xpath_coordinates = '//div[@class = "Z0LcW"]'
dataset_coordinates = pd.DataFrame(columns=["STATE","DISTRICT","COORDINATES"])
dataset_coordinates_existing = pd.read_csv(r"C:\Users\mangl\Desktop\python_projects\project_2\district_coordinates.csv")

for place in dataset_dist.itertuples():
    #print(place.STATE,',', place.DISTRICT, 'Coordinates')
    if (place.STATE not in dataset_coordinates_existing["STATE"].unique().tolist()) or (place.STATE in dataset_coordinates_existing["STATE"].unique().tolist() and place.DISTRICT not in dataset_coordinates_existing[dataset_coordinates_existing["STATE"] == place.STATE]["DISTRICT"].tolist()):
        try : 
        #print(place.STATE,',', place.DISTRICT, 'Coordinates')
            search = driver.find_elements_by_xpath(xpath_search)
            search = search[0]
            search.clear()
            search.send_keys(place.STATE,',', place.DISTRICT, ' Coordinates')
            click = driver.find_element_by_xpath(xpath_click)
            click.click()
            coordinates = driver.find_element_by_xpath(xpath_coordinates).text
            #print(place.STATE,',', place.DISTRICT, 'Coordinates: ',coordinates)
            record = {"STATE":place.STATE, "DISTRICT":place.DISTRICT, "COORDINATES":coordinates}
            dataset_coordinates = dataset_coordinates.append(record, ignore_index=True)
            driver.back()
        except NoSuchElementException:
            record = {"STATE":place.STATE, "DISTRICT":place.DISTRICT, "COORDINATES":''}
            dataset_coordinates = dataset_coordinates.append(record, ignore_index=True)
            driver.back()
            print("No element found")

driver.close()
time_str = datetime.now().strftime('%Y%m%d%H%M%S')
name = "district_coordinates"+time_str+'.csv'
dataset_coordinates.to_csv(name,index=False)
print("{} is generated in path {}".format(name,os.getcwd()))

district_coordinates20200422174953.csv is generated in path C:\Users\mangl\Desktop\python_projects\project_2
